In [1]:
%run pandas_implementation

In [6]:
%run vaex_implementation_v1

In [7]:
import ipytest
ipytest.autoconfig()

In [9]:
%%run_pytest[clean] -s
# Testing roundtrip
# data -> Vaex -> Vaex (from_dataframe_to_vaex) -> Pandas (from_dataframe) -> Vaex (from_dataframe_to_vaex)

def test_roundtrip():
    
    # Test data
    x = np.array([1.5, 2.5, 3.5])
    y = np.array([9.2, 10.5, 11.8])
    df = vaex.from_arrays(x=x, y=y)
    
    print("Data: Vaex and pointer")
    b0 = df.x.to_numpy().__array_interface__['data'][0]
    
    # Step 1
    print("Step 1: Vaex (from_dataframe_to_vaex) and pointer")
    df1 = from_dataframe_to_vaex(df)
    b1 = df1.x.to_numpy().__array_interface__['data'][0]
    
    print(df1)
    print(b1)
    
    # Step 2
    print("Step 2: Pandas (from_dataframe) and pointer")
    df2 = from_dataframe(df1)
    b2 = df2.x.to_numpy().__array_interface__['data'][0]
    
    print(df2)
    print(b2)
    
    # Step 3
    print("Step 3: Vaex (from_dataframe_to_vaex) and pointer")
    df3 = from_dataframe_to_vaex(df2)
    b3 = df3.x.to_numpy().__array_interface__['data'][0]
    
    print(df3)
    print(b3)
    
    assert b0 == b3

Data: Vaex and pointer
Step 1: Vaex (from_dataframe_to_vaex) and pointer
  #    x     y
  0  1.5   9.2
  1  2.5  10.5
  2  3.5  11.8
2042418436992
Step 2: Pandas (from_dataframe) and pointer
     x     y
0  1.5   9.2
1  2.5  10.5
2  3.5  11.8
2042418436992
Step 3: Vaex (from_dataframe_to_vaex) and pointer
  #    x     y
  0  1.5   9.2
  1  2.5  10.5
  2  3.5  11.8
2042418436992
.
1 passed in 0.01s


In [11]:
%%run_pytest[clean] -s
# Cheking value changes
# See if it affects the other object also
# (if the buffer is the same, data should be changed on both df and df2

def test_value_change():
    
    # Test data
    x = np.array([1.5, 2.5, 3.5])
    y = np.array([9.2, 10.5, 11.8])
    df = vaex.from_arrays(x=x, y=y)
    
    # Step 1
    df1 = from_dataframe_to_vaex(df)
    
    # Step 2
    df2 = from_dataframe(df1)
    
    # Step 3
    df3 = from_dataframe_to_vaex(df2)
    
    print("\nDataframes before value change:\n")
    print(df)
    print(df3)

    print("\nAfter value change\n")
    df['x'] = df.func.where(df.x == 1.5, 333, df.x)
    print(df)
    print(df3)

    print("\nAfter another value change\n")
    df3['y'] = df3.func.where(df3.y == 11.8, 777, df3.y)
    print(df)
    print(df3)


Dataframes before value change:

  #    x     y
  0  1.5   9.2
  1  2.5  10.5
  2  3.5  11.8
  #    x     y
  0  1.5   9.2
  1  2.5  10.5
  2  3.5  11.8

After value change

  #      x     y
  0  333     9.2
  1    2.5  10.5
  2    3.5  11.8
  #    x     y
  0  1.5   9.2
  1  2.5  10.5
  2  3.5  11.8

After another value change

  #      x     y
  0  333     9.2
  1    2.5  10.5
  2    3.5  11.8
  #    x      y
  0  1.5    9.2
  1  2.5   10.5
  2  3.5  777
.
================================================== warnings summary ===================================================
tmp3gafiquy.py: 16 warnings
  C:\Users\Alenka\anaconda3\envs\df_protocol\lib\ast.py:407: DeprecationWarning: visit_Num is deprecated; add visit_Constant
    return visitor(node)

-- Docs: https://docs.pytest.org/en/stable/warnings.html
1 passed, 16 warnings in 0.03s
